In [1]:
!pip install keras


In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D,Conv2DTranspose,Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


In [3]:
def conv_block(x, n_base, batch_normalization):
    
    x = Conv2D(filters=n_base, kernel_size=(3,3), 
                        strides=(1,1),padding='same')(x)
    if (batch_normalization):
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=n_base, kernel_size=(3,3), 
                        strides=(1,1),padding='same')(x)
    if (batch_normalization):
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

In [4]:
def downsample_block(x, n_base, batch_normalization, dropout):
    f = conv_block(x, n_base, batch_normalization)
    p = layers.MaxPool2D(pool_size = (2,2))(f)
    if(dropout):
        p = layers.Dropout(0.2)(p)
        
    return f, p

In [5]:
def upsample_block(x, f, n_base, batch_normalization, dropout):
    
    x = Conv2DTranspose(filters=n_base, kernel_size=(2,2), 
                         strides=(2,2),padding='same')(x)
    x = Concatenate()([x,f])
    if(dropout):
        x = layers.Dropout(0.2)(x)
    x = conv_block(x, n_base, batch_normalization)
        
    return x

In [6]:
def get_unet_weightmap(img_w, img_h, img_ch, n_base, LR, batch_normalization, dropout):

    
    
    ## Encoder part
#     model = Sequential()
    image = layers.Input((img_w, img_h, img_ch))
    weight = layers.Input((img_w, img_h, img_ch))
    inputs= layers.concatenate([image,weight],axis=-1)

    # inputs = layers.Input((img_w, img_h, img_ch))
    
    f1, p1 = downsample_block(inputs, n_base, batch_normalization, dropout)
    f2, p2 = downsample_block(p1, n_base*2, batch_normalization, dropout)
    f3, p3 = downsample_block(p2, n_base*4, batch_normalization, dropout)
    f4, p4 = downsample_block(p3, n_base*8, batch_normalization, dropout)
    
    
    ## Bottleneck
    bottleneck = conv_block(p4, n_base*16, batch_normalization)
    
    ## Decoder part
    p5 = upsample_block(bottleneck, f4, n_base*8, batch_normalization, dropout)
    p6 = upsample_block(p5, f3, n_base*4, batch_normalization, dropout)
    p7 = upsample_block(p6, f2, n_base*2, batch_normalization, dropout)
    p8 = upsample_block(p7, f1, n_base, batch_normalization, dropout)

    
    ## 1 Convo layer
    p9 = Conv2D(filters=1, kernel_size=(1,1), 
                            padding='same')(p8)
    outputs = Activation('sigmoid')(p9)
    

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.summary()
    
    return model


In [7]:
!pip install scikit-image

In [8]:
## Task1a) Lung segmentation in chest X-ray images:
import os
from random import shuffle
from skimage.io import imread
from skimage.transform import resize


image_path = '/DL_course_data/Lab3/MRI/Image' 
mask_path = '/DL_course_data/Lab3/MRI/Mask'



In [9]:
def load_data(image_path,mask_path):
    
    image_list = os.listdir(image_path)
    mask_list = os.listdir(mask_path)
    
    images = []
    masks = []

    for image in image_list:
        img = imread(os.path.join(image_path, image), as_gray=True)  # "as_grey"
        img = resize(img, (240, 240), anti_aliasing=True).astype('float32')
        images.append(img)

    for image in image_list:
        mask = str.replace(image,'.png','_Tumor.png')
        mask_img = imread(os.path.join(mask_path, mask), as_gray=True)
        mask_img = resize(mask_img, (240, 240), anti_aliasing=True).astype('float32')
        masks.append(mask_img)
        
    ## Load data in traditional way
    # img_train, img_val, mask_train, mask_val = train_test_split(images, masks, shuffle = True,
    #                                                   test_size = 0.2)

    images = np.expand_dims(images, axis = -1)
    masks = np.expand_dims(masks, axis = -1)
    
    return np.array(images), np.array(masks)
    

In [10]:
from tensorflow.keras import backend as K
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 0.0001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.0001)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [11]:
n_base =8
LR = 1e-4
batch_normalization = True
dropout = True
epochs = 100
batch_size = 8
weight_strength = 1

img_w, img_h = 240,240
img_ch = 1

In [12]:
images, masks = load_data(image_path,mask_path)

In [13]:
from scipy.ndimage import binary_dilation, binary_erosion

weight_boundary = []

radius = 2
structure = np.ones((5,5))


for i in range(len(masks)):
    weight_boundary.append(binary_dilation(masks[i,:,:,0], structure).astype(int) - binary_erosion(masks[i,:,:,0],structure).astype(int))
    # mask_eroded[i,:,:,0] = binary_erosion(masks[i,:,:,0],structure)
    # # weight_boundary[i,:,:,0] = mask_dilated[i,:,:,0] - mask_eroded[i,:,:,0]

In [17]:
import gc
gc.collect()

2136

In [19]:
def weighted_loss(weight_map, weight_strength): 
    def weighted_dice_loss(y_true, y_pred):
        y_true_f = K.flatten(y_true) 
        y_pred_f = K.flatten(y_pred)
        weight_f = K.flatten(weight_map) 
        weight_f = weight_f * weight_strength 
        weight_f = 1/(weight_f + 1)
        weighted_intersection = K.sum(weight_f * (y_true_f * y_pred_f))
        return -(2. * weighted_intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
    return weighted_dice_loss

In [20]:
from numpy import concatenate

def generator_with_weights(x_train, y_train, weight_train, batch_size):
    while True:
               
        for ind in (range(0, len(x_train), batch_size)):
            
            batch_img = x_train[ind:ind+batch_size]
            batch_weightmap = weight_train[ind:ind+batch_size]
            batch_label = y_train[ind:ind+batch_size]
            
            # Sanity check assures batch size always satisfied
            # by repeating the last 2-3 images at last batch.
            length = len(batch_img)
            if length == batch_size:
                pass
            else:
                for tmp in range(batch_size - length):
                    batch_img = np.append(batch_img, np.expand_dims(batch_img[-1],axis=0), axis = 0)
                    batch_weightmap = np.append(batch_weightmap, np.expand_dims(batch_weightmap[-1],axis=0), axis = 0)
                    batch_label = np.append(batch_label, np.expand_dims(batch_label[-1], axis=0), axis = 0)
        
            backgound_value = x_train.min()
            data_gen_args = dict(rotation_range=10.,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     cval = backgound_value,
                                     zoom_range=0.2,
                                     horizontal_flip = True)
            
            image_datagen = ImageDataGenerator(**data_gen_args)
            weights_datagen = ImageDataGenerator(**data_gen_args)
            mask_datagen = ImageDataGenerator(**data_gen_args)
            
            image_generator = image_datagen.flow(batch_img, shuffle=False,
                                                 batch_size=batch_size,
                                                 seed=1)

            weights_generator = image_datagen.flow(batch_weightmap, shuffle=False,
                                                 batch_size=batch_size,
                                                 seed=1)
            
            mask_generator = mask_datagen.flow(batch_label, shuffle=False,
                                               batch_size=batch_size,
                                               seed=1)
            
            image = image_generator.next()
            weight = weights_generator.next()
            label = mask_generator.next()
            input = concatenate([image,weight],axis=-1)
            
            
            yield input, label
            # yield (image, label)

In [20]:
from numpy import concatenate

def n_generator_with_weights(x_train, y_train, weight_train, batch_size):
    while True:
               
        for ind in (range(0, len(x_train), batch_size)):
            
            batch_img = x_train[ind:ind+batch_size]
            batch_weightmap = weight_train[ind:ind+batch_size]
            batch_label = y_train[ind:ind+batch_size]
            
            # Sanity check assures batch size always satisfied
            # by repeating the last 2-3 images at last batch.
            length = len(batch_img)
            if length == batch_size:
                pass
            else:
                for tmp in range(batch_size - length):
                    batch_img = np.append(batch_img, np.expand_dims(batch_img[-1],axis=0), axis = 0)
                    batch_weightmap = np.append(batch_weightmap, np.expand_dims(batch_weightmap[-1],axis=0), axis = 0)
                    batch_label = np.append(batch_label, np.expand_dims(batch_label[-1], axis=0), axis = 0)
        
            backgound_value = x_train.min()
            data_gen_args = dict(rotation_range=10.,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     cval = backgound_value,
                                     zoom_range=0.2,
                                     horizontal_flip = True)
            
            image_datagen = ImageDataGenerator(**data_gen_args)
            weights_datagen = ImageDataGenerator(**data_gen_args)
            mask_datagen = ImageDataGenerator(**data_gen_args)
            
            image_generator = image_datagen.flow(batch_img, shuffle=False,
                                                 batch_size=batch_size,
                                                 seed=1)

            weights_generator = image_datagen.flow(batch_weightmap, shuffle=False,
                                                 batch_size=batch_size,
                                                 seed=1)
            
            mask_generator = mask_datagen.flow(batch_label, shuffle=False,
                                               batch_size=batch_size,
                                               seed=1)
            
            image = image_generator.next()
            weight = weights_generator.next()
            label = mask_generator.next()
            # input = concatenate([image,weight],axis=-1)
            
            
            yield concatenate([image,weight],axis=-1), label
            gc.collect()
            # yield (image, label)

In [21]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4)
cvscores = []


In [24]:
def show_hist(History): 
    plt.figure(figsize=(4, 4))
    plt.title("Learning curve")
    plt.plot(History.history["loss"], label="loss")
    plt.plot(History.history["val_loss"], label="val_loss")
    plt.plot( np.argmin(History.history["val_loss"]),
            np.min(History.history["val_loss"]),
            marker="x", color="r", label="best model")

    plt.xlabel("Epochs")
    plt.ylabel("Loss Value")
    plt.legend()
    plt.show()

    plt.figure

    plt.figure(figsize=(4, 4))
    plt.title("Accuracy")
    plt.plot(History.history["accuracy"], label="accuracy")
    plt.plot(History.history["val_accuracy"], label="val_accuracy")
    plt.plot( np.argmax(History.history["val_accuracy"]),
            np.max(History.history["val_accuracy"]),
            marker="x", color="r", label="best model")

    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

In [24]:
from matplotlib import image

for train, val in kf.split(images, masks, weight_boundary):
    # image_train, image_val = images[train],images[val]
    # weight_train, weight_val = weight_boundary[train], weight_boundary[val]
    # mask_train, mask_val = masks[train], masks[val]
    ## Get model
    model = get_unet_weightmap(img_w, img_h, img_ch, n_base, LR, 
                batch_normalization, dropout)

    # train_generator = generator_with_weights(images[train], masks[train], weight_boundary[train], batch_size)
    # val_generator = generator_with_weights(images[val], masks[val], weight_boundary[val], batch_size)
    
    # Compile the model
    model.compile(loss = [dice_coef_loss],          # Model Compiling   
                optimizer = Adam(lr = LR),
                metrics = [dice_coef, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    model_history = model.fit_generator(generator_with_weights(images[train], masks[train], weight_boundary[train], batch_size),
        steps_per_epoch = len(images[train])//batch_size,
        validation_data = generator_with_weights(images[val], masks[val], weight_boundary[val], batch_size), 
        validation_steps = len(images[val])//batch_size,
        epochs = epochs,  verbose=1)
    show_hist(model_history)

    # evaluate the model
    scores = model.evaluate(images[val], masks[val], verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 240, 240, 2  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 240, 240, 8)  152         ['input_3[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 240, 240, 8)  32         ['conv2d[1][0]']                 
 alization)                                                                                       
                                                                                              

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


: 

: 

In [ ]:
cvscores

[2.9252661392092705, 10.042130947113037, 4.18890155851841, 3.611527383327484]

In [ ]:
model.metrics_names

['loss', 'dice_coef', 'precision_8', 'recall_8']